In [1]:
%pip install h2o

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 177.0 MB 131.3 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 91.5 MB/s eta 0:00:01
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=0533030546660b1c148809ab4577dcff647b0b881df5f2d4b3bf02c973cf12d2
  Stored in directory: /home/mk7516/.cache/pip/wheels/bf/d7/e0/905f059465fafaf07c9e79ba4c8d642ac646b01ea8c38b07d5
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=4f831c65e1c94dc6bcb6bd66006a00f5a1ba67ae0b30925cbe353bfd64aac98a
  Stored in directory: /home/mk7516/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built h2o future
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19156485/tmpigv62yo0
  JVM stdout: /state/partition1/job-19156485/tmpigv62yo0/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19156485/tmpigv62yo0/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_mk7516_bbu27h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.543 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup('airlines')

/home/mk7516/.local/lib/python3.8/site-packages/sklearn/datasets/_openml.py:417: UserWarning: Multiple active versions of the dataset matching the name airlines exist. Versions may be fundamentally different, returning version 1.
  warn(


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(404537, 8)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

16:30:45.470: GBM_lr_annealing_selection_AutoML_1_20220506_153040 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_4_AutoML_1_20220506_153040

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.18526880737136234
RMSE: 0.4304286321463319
LogLoss: 0.5503749008098437
Null degrees of freedom: 10135
Residual degrees of freedom: 10122
Null deviance: 13906.059081546015
Residual deviance: 11157.199989217152
AIC: 11185.199989217152
AUC: 0.7947937805957815
AUCPR: 0.8228233820429297
Gini: 0.5895875611915631

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.43708662308864593: 


,,0,1,Error,Rate
0,0,2085.0,2374.0,0.5324,(2374.0/4459.0)
1,1,559.0,5118.0,0.0985,(559.0/5677.0)
2,Total,2644.0,7492.0,0.2894,(2933.0/10136.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.437087,0.777280,253.0
1,max f2,0.204436,0.875616,348.0
2,max f0point5,0.556186,0.755454,190.0
3,max accuracy,0.514363,0.728295,212.0
4,max precision,0.951518,1.000000,0.0
5,max recall,0.100610,1.000000,387.0
6,max specificity,0.951518,1.000000,0.0
7,max absolute_mcc,0.514363,0.443768,212.0
8,max min_per_class_accuracy,0.567156,0.717650,183.0
9,max mean_per_class_accuracy,0.556186,0.721017,190.0



Gains/Lift Table: Avg response rate: 56.01 %, avg score: 55.38 %


...


Gains/Lift Table: Avg response rate: 55.45 %, avg score: 55.45 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010002,0.896072,1.660003,1.660003,0.920415,0.912948,0.920415,0.912948,0.016603,0.016603,66.000309,66.000309,0.014816
1,2,0.020001,0.875744,1.601113,1.630562,0.887763,0.885255,0.904091,0.899103,0.016010,0.032612,60.111300,63.056168,0.028307
2,3,0.030002,0.860037,1.554358,1.605159,0.861839,0.867486,0.890006,0.888563,0.015546,0.048158,55.435843,60.515851,0.040751
3,4,0.040001,0.847717,1.540921,1.589101,0.854388,0.853648,0.881102,0.879836,0.015408,0.063566,54.092078,58.910106,0.052891
4,5,0.050000,0.837023,1.515060,1.574294,0.840049,0.842217,0.872893,0.872313,0.015149,0.078715,51.506042,57.429439,0.064451
5,6,0.100001,0.794311,1.462036,1.518165,0.810649,0.814542,0.841771,0.843427,0.073102,0.151818,46.203586,51.816513,0.116303
6,7,0.150001,0.760267,1.399442,1.478591,0.775943,0.776837,0.819828,0.821231,0.069973,0.221790,39.944215,47.859080,0.161131
7,8,0.200001,0.730155,1.324276,1.440012,0.734266,0.744922,0.798438,0.802154,0.066214,0.288005,32.427619,44.001215,0.197523
8,9,0.300000,0.676905,1.264522,1.381517,0.701135,0.702923,0.766004,0.769077,0.126450,0.414455,26.452242,38.151654,0.256894
9,10,0.400000,0.628146,1.189013,1.333390,0.659267,0.652223,0.739320,0.739863,0.118902,0.533357,18.901313,33.339039,0.299318




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.652005,0.002269,0.653217,0.653438,0.650675,0.654033,0.648661
1,auc,0.724865,0.001526,0.722839,0.725505,0.726868,0.725103,0.724011
2,err,0.347995,0.002269,0.346783,0.346562,0.349325,0.345967,0.351339
3,err_count,28154.800000,118.702150,28259.000000,28111.000000,28193.000000,27969.000000,28242.000000
4,f0point5,0.669435,0.002513,0.671293,0.670331,0.667341,0.671967,0.666241
5,f1,0.746087,0.001152,0.745352,0.745931,0.746714,0.747688,0.744749
6,f2,0.842584,0.003657,0.837780,0.840749,0.847517,0.842643,0.844231
7,lift_top_group,1.658288,0.015779,1.642743,1.681097,1.663181,1.660461,1.643958
8,logloss,0.603542,0.001020,0.605007,0.603004,0.602381,0.603237,0.604081
9,max_per_class_error,0.684097,0.011247,0.672062,0.675693,0.696384,0.680883,0.695466



See the whole table with table.as_data_frame()


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_4_AutoML_1_20220506_153040,0.724861,0.603544,0.74801,0.380846,0.456444,0.208341
StackedEnsemble_AllModels_3_AutoML_1_20220506_153040,0.724854,0.603546,0.747967,0.378007,0.456445,0.208342
StackedEnsemble_BestOfFamily_4_AutoML_1_20220506_153040,0.723891,0.604322,0.747064,0.380831,0.456803,0.208669
StackedEnsemble_AllModels_2_AutoML_1_20220506_153040,0.723554,0.604556,0.746497,0.37861,0.456918,0.208774
StackedEnsemble_AllModels_1_AutoML_1_20220506_153040,0.723399,0.604625,0.746361,0.378472,0.456954,0.208807
StackedEnsemble_BestOfFamily_3_AutoML_1_20220506_153040,0.722711,0.605203,0.745649,0.380098,0.457222,0.209052
StackedEnsemble_BestOfFamily_1_AutoML_1_20220506_153040,0.722547,0.605294,0.7454,0.378875,0.457256,0.209083
StackedEnsemble_BestOfFamily_2_AutoML_1_20220506_153040,0.722496,0.605327,0.745507,0.375091,0.457283,0.209108
XGBoost_grid_1_AutoML_1_20220506_153040_model_9,0.721734,0.605943,0.745071,0.383226,0.457562,0.209363
GBM_1_AutoML_1_20220506_153040,0.721615,0.606003,0.744623,0.383026,0.457596,0.209394
